In [1]:
import numpy as np

1) Сравните метрики hit_rate@k, precision@k. Какую матрику использовать предпочтительно и почему. Привидите пример 2-3 задач (опишите, что является клиентом, что товаром), в которой более уместно использовать метрику hit_rate?

hit_rate@k - был ли хотя бы 1 релевантный товар среди рекомендованных 

precision@k - доля релевантных товаров среди k рекомендованных = Какой % рекомендованных товаров юзер купил

1. Рекомендация музыки на яндексмузыке. Предпочтительней precision@k.
музыка - товар, юзер - клиент.

2. Рекомендация товаров ситилинк. Предпочтительней hit_rate@k. быт. ьехника - товар, юзер - клиент.

2)В метрике NDCG@k мы используем логарифм в знаменателе. Как Вы думаете, почему именно логарифм? Какую функцию можно использовать вместо логарифма? Привидите пример метрик/подходов к предобработке данных/функций ошибок в ML, где также в знаменателе присутствует логарифм.

Использование логарифма как функции дисконтирования можно объяснить следующими интуитивными соображениями: с точки зрения ранжирования позиции в начале списка отличаются гораздо сильнее, чем позиции в его конце. Так, в случае поискового движка между позициями 1 и 11 целая пропасть (лишь в нескольких случаях из ста пользователь заходит дальшей первой страницы поисковой выдачи), а между позициями 101 и 111 особой разницы нет — до них мало кто доходит. Эти субъективные соображения прекрасно выражаются с помощью логарифма.

Вместо логарифма можно выбрать любую медленно возрастающую функцию, напрмер корень из x.

Логарифм в знаменателе нашел только у Discounted cumulative gain at K (DCG@K) и normalized discounted cumulative gain at K (nDCG@K)

3)Какие еще матрики (Вы можете вспомнить уже пройденные Вами или посмотреть в интернете) могут использоваться для рекомендательных систем (приведите примеры метрики и чем являются интеракции, чтобы она могла быть использована).ь

Cumulative gain at K (CG@K) — базовая метрика ранжирования, которая использует простую идею: чем релевантные элементы в этом топе, тем лучше.

Expected reciprocal rank (ERR) — пример метрики качества ранжирования, основанной на каскадной модели.

Метрики на основе ранговой корреляции

Ранговый коэффициент корреляции Кендэлла: основан на подсчете согласованных (и несогласованных) пар у перестановок — пар элементов, котором перестановки присвоили одинаковый (разный) порядок.

Ранговый коэффициент корреляции Спирмена: корреляция Пирсона, посчитанная на значениях рангов.

4)boughted = [1, 3, 5, 7, 9, 11]

recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]

Посчитайте на этих данных pr@8, rec@8, AP@8, NDCG@8, RR@8, ERR@8

In [2]:
boughted = [1, 3, 5, 7, 9, 11]
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]

In [3]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    print(flags)
    
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

precision_at_k(recommended, boughted, k=8)

[False False  True  True  True  True]


0.5

In [4]:
def recall_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

recall_at_k(recommended, boughted, k=8)

0.6666666666666666

In [5]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(1, k+1): 
        if flags[i] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i)
            sum_ += p_k
            
    result = sum_ / sum(flags)
    
    return result

ap_k(recommended, boughted, k=8)

[False False False False False False]
[False False  True False False False]
[False False  True  True False False]
[False False  True  True False  True]


0.32

In [6]:
def nDCG_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    if recommended_list.shape[0] < k:
        k = recommended_list.shape[0]
    
    product_availability = [(recommended_list[i] in bought_list) for i in range(k)]
    
    #DCG@k
    DCG = 0
    DCG += product_availability[0] / 1
    for i in range(1, k):
        DCG += product_availability[i] / np.log(i+1)
    DCG = DCG / k
    
    #iDCG
    iDCG = 1
    for i in range(1, k):
        iDCG += 1 / np.log(i+1)
    iDCG = iDCG / k
    
    return DCG / iDCG
    
nDCG_k(recommended, boughted, k=8)      

0.5653142737255068

In [7]:
def mean_reciprocal_rank(recommended_list, bought_list):
    ranks=0.
    for item_rec in recommended_list:
        for i, item_bought in enumerate(bought_list):
            if item_rec == item_bought:
                ranks += 1 / (i+1)
    return ranks / len(recommended_list)

mean_reciprocal_rank(recommended, boughted)    

0.145

In [8]:
def reciprocal_rank(recommended_list, bought_list):
    ranks=0.
    
    for i, item in enumerate(recommended_list):
        if item in bought_list:
            return 1 / (i+1)
    return 0
    
reciprocal_rank(recommended, boughted)   

0.5

In [9]:
def expected_reciprocal_rank(recommended_list, bought_list, k=8):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    item_relevant = [(recommended_list[i] in bought_list) for i in range(recommended_list.shape[0])]
    print(item_relevant)
    
    rank_item_relevant = [i+1 for i in range(len(item_relevant)) if item_relevant[i]]    
    
    max_r_true = max(rank_item_relevant)
    print(rank_item_relevant)
    
    p_k = np.zeros(k)    
    for i in range(k):
        p_k[i] = (2**item_relevant[i] - 1) / (2**1 - 1)   
    print(p_k)
    
    P_k = np.zeros(k)    
    P_k[0] = p_k[0]
    for i in range(1, k):
        P_k[i] = p_k[i]
        for j in range(1, i):
            P_k[i] *= (1 - p_k[j]) 
        P_k[i] = P_k[i] / k
    print(P_k)
        
    return sum(P_k)

expected_reciprocal_rank(recommended, boughted, k=8)   

[False, True, True, False, True, True, False, False]
[2, 3, 5, 6]
[0. 1. 1. 0. 1. 1. 0. 0.]
[0.    0.125 0.    0.    0.    0.    0.    0.   ]


0.125